In [ ]:
import json
import requests
from openai import OpenAI
from inference import (base_url, inflection_api_key)

# **Function Calling with Structured API Queries**

This notebook demonstrates how to implement **function calling** using new OpenAI compatible API for structured API queries. The goal is to efficiently retrieve relevant information based on specific user inputs by utilizing structured function calls. This approach enhances **accuracy**, improves **response efficiency**, and enables seamless **integration** with various API-based workflows.

## **Overview**

This notebook focuses on structured function execution to enhance automation and data retrieval. It provides a systematic approach to invoking API functions with well-defined parameters.

## **Key Functionalities**

This notebook implements the following function:

- `get_weather(latitude: str, longitude: str) -> dict:`  
  - Fetches weather data based on geographic coordinates.

The structured approach ensures that API calls are **consistent, interpretable, and easily adaptable** for various use cases.


In [ ]:
url = base_url + "/external/api/inference/openai/v1/"
model = "inflection_3_with_tools"
client = OpenAI(base_url=url, api_key=inflection_api_key)

In [ ]:
def get_weather(latitude: str, longitude: str) -> dict:
    """
    Get the weather information for the given latitude and longitude.
    """
    try:
        # Fetch the weather data in Celsius
        response_celsius = requests.get(
            f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}"
            f"&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
            f"&temperature_unit=celsius"
        )
        data_celsius = response_celsius.json()
        temperature_celsius = data_celsius['current']['temperature_2m']

        # Fetch the weather data in Fahrenheit
        response_fahrenheit = requests.get(
            f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}"
            f"&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
            f"&temperature_unit=fahrenheit"
        )
        data_fahrenheit = response_fahrenheit.json()
        temperature_fahrenheit = data_fahrenheit['current']['temperature_2m']

        return {
            'temperature_celsius': temperature_celsius,
            'temperature_fahrenheit': temperature_fahrenheit
        }
    except Exception as e:
        return {"error": str(e)}

In [ ]:
# Honolulu, Hawaii
latitude = "21.3069"
longitude = "-157.8583"
weather_data = get_weather(latitude, longitude)
print(weather_data)

In [ ]:
tool_list = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather for a given location",
            "parameters": {
                "properties": {
                    "latitude": {
                        "type": "string", 
                        "description": "Latitude of the location"
                    },
                    "longitude": {
                        "type": "string", 
                        "description": "Longitude of the location"
                    }
                },
                "required": ["latitude", "longitude"],
                "type": "object",
                "additionalProperties": False,
            },
        },
    }
]

In [ ]:
system_message = {
    "role": "system",
    "content": "You are a helpful assistant. When calling a tool, use the tool results in your response. Use emojis to make your response more engaging."
}

# Define messages for user prompts
messages = [
    system_message,
    {
        "role": "user",
        "content": "What is the weather like in Honolulu, Hawaii?"
    },
]

In [ ]:
# First API call to determine if function calling is needed
completion = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tool_list,
    stream=False
)

print(f"\nUser Input: {messages[-1]}")
print(f"\nFirst Assistant Response: {completion.choices[0].message}")

In [ ]:
class color:
    BOLD = '\033[1m'
    END = '\033[0m'

In [ ]:
if completion.choices[0].message.tool_calls:
    # Extract function details
    tool_call = completion.choices[0].message.tool_calls[0]
    print(f"\n{color.BOLD}Tool Call: {color.END} {tool_call}")
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    # Execute the function call
    result = eval(f"{function_name}('{arguments['latitude']}', '{arguments['longitude']}')")
    print(f"\n{color.BOLD}Tool Response: {color.END} {result}")


    # Append the tool call result as a **string**
    messages.append({
        "role": "tool",
        "content": str(result),
        "tool_call_id": completion.choices[0].message.tool_calls[0].id
    })
    print(f"\n{color.BOLD}Tool Messages: {color.END} {messages}")

    
    # Final call to generate response with function output
    completion_2 = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tool_list,
        stream=False,
    )

    print(f"\n{color.BOLD}Final Assistant Response: {color.END} {completion_2.choices[0].message.content}")